In [39]:
'''
This notebook is to be used to convert best-fit parameter values saved 
in climatools.lblnew.bestfit_params into Fortran, in a form that can be 
easily inserted into clirad-lw.  
'''

'\nThis notebook is to be used to convert best-fit parameter values saved \nin climatools.lblnew.bestfit_params into Fortran, in a form that can be \neasily inserted into clirad-lw.  \n'

In [40]:
import os
import itertools
import importlib

from climatools.lblnew import bestfit_params as bestfit
from climatools.cliradlw import setup as setup_cliradlw

importlib.reload(bestfit)
importlib.reload(setup_cliradlw)

<module 'climatools.cliradlw.setup' from '/chia_cluster/home/jackyu/climatools/climatools/cliradlw/setup.py'>

In [41]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    return {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}

In [42]:
def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4, dtype=float):
    '''
    '''
    if dtype == float:
        strfmt = '{:15.6e}'
    elif dtype == int:
        strfmt = '{:15d}'
    else:
        raise ValueError('dtype must be either float or int.')
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines


def vector_to_F77(array=None, num_values_per_line=None, dtype=None):
    lines = vector_to_F77list(array=array, 
                              num_values_per_line=num_values_per_line,
                              dtype=dtype)
    
    rlines = [5 * ' ' + '&' + l for l in lines]
    
    fortran = '\n'.join(rlines)
    return fortran

In [43]:
def comment_header(param):
    s = "! {} band{}"
    return s.format(param['molecule'], param['band'])


def ng(param):
    s = "ng = {:d}"
    return s.format(sum(param['ng_refs']))


def wgt(param):
    vs = [v for ref in param['wgt'] for v in ref]
    s = vector_to_F77(vs, 
                      num_values_per_line=3, dtype=float)
    ls = ['wgt(1:ng) = (/',
          s,
          5 * ' ' + '&' + '/)']
    return '\n'.join(ls)


def w_diffuse(param):
    vs = [v for ref in param['w_diffuse'] for v in ref]
    s = vector_to_F77(vs, 
                      num_values_per_line=3, dtype=float)
    ls = ['w_diffuse(1:ng) = (/',
          s,
          5 * ' ' + '&' + '/)']
    return '\n'.join(ls)

In [44]:
def gasband_str_funcs():
    return (comment_header,
            ng,
            wgt,
            w_diffuse,)

In [45]:
def kdist_param_gasband(param):
    '''
    Returns list of strings for some gas and band.
    '''
    print(param['molecule'], param['band'])
    return [f(param) for f in gasband_str_funcs()]


def kdist_param_gas(params):
    '''
    Returns list of strings for some gas.
    '''
    molecules = [param['molecule'] for param in params]
    try:
        assert all([molecule == molecules[0] for molecule in molecules])
    except AssertionError:
        raise('All input param dicts should be for the same gas.')
        
    ls_gas = []
    for i, param in enumerate(params):
        if i == 0:
            s_if = 'if (ib == {}) then'
        else:
            s_if = 'else if (ib == {}) then'
        
        s_if = s_if.format(band_map()[param['band']])
                 
        ls = kdist_param_gasband(param)
        ls = [3 * ' ' + l for l in ls]
        ls = [s_if] + ls
        
        ls_gas.extend(ls)
       
    s = "write (*, *) 'k-dist bestfits unavailable for {} band', ib"
    s = s.format(molecules[0].upper())
    ls_else = []
    ls_else.append(s)
    ls_else.append('stop')
    ls_else = [3 * ' ' + l for l in ls_else]
    
    ls_gas.append('else')
    ls_gas.extend(ls_else)
    ls_gas.append('end if')      
    return ls_gas


def gas2mid(gas):
    d = {'h2o': 1, 'co2': 2, 'o3': 3, 'n2o': 4, 'ch4': 5, 'o2':6}
    return d[gas]


def kdist_param():
    'Returns list of strings covering all gases and their bands'
    gasband_gs = [h2o_gasbands(), co2_gasbands(), o3_gasbands(),
                  n2o_gasbands(), ch4_gasbands()]
    
    lines = []
    for i, gasbands in enumerate(gasband_gs):
        
        params = [bestfit.kdist_params(molecule=gas, band=band)
                  for gas, band in gasbands]
        
        gas = params[0]['molecule']
        mid = gas2mid(gas)
        
        if i == 0:
            s_if = 'if (mid == {}) then'
        else:
            s_if = 'else if (mid == {}) then'
        s_if = s_if.format(mid)
        
        ls = kdist_param_gas(params)
        ls = [3 * ' ' + l for l in ls]
        ls = [s_if] + ls
        
        lines.extend(ls)
    
    s = "write (*, *) 'k-dist bestfits unavailable for gas id:', mid"
    s = s.format(mid)
    ls_else = []
    ls_else.append(s)
    ls_else.append('stop')
    ls_else = [3 * ' ' + l for l in ls_else]
    
    lines.append('else')
    lines.extend(ls_else)
    lines.append('end if')     
    return lines


def subroutine():
    ls = ('subroutine get_kdist_bestfits(mid, ib, ng, wgt, w_diffuse)',
          '!     Get the lblnew bestfit parameters',
          '',
          'implicit none',
          '',
          'integer, parameter :: max_ng = 15  ! max number of g-interval allowed'
          'integer, parameter :: npt = 295  ! max number of (p, t) in store k-table',
          '',
          'integer :: mid ! gas id',
          'integer :: ib  ! spectral band number',
          'integer :: ng ! number of g-intervals', 
          'real :: wgt(max_ng)',
          'real :: w_diffuse(max_ng)',
          'real :: dgs(max_ng)    ! Planck-weighted k-distribution function')
    
    lines = list(ls)
    lines = lines + kdist_param()
    lines.append('return')
    lines.append('end')
    return lines


def file_content():
    lines = subroutine()
    lines = [6 * ' ' + l for l in lines]
    s = '\n'.join(lines)
    return s


def script():
    s = file_content()
    
    fpath = os.path.join(setup_cliradlw.DIR_SRC, 'kdist_bestfits.f')
    with open(fpath, mode='w', encoding='utf-8') as f:
        f.write(s)

In [46]:
# These are the gases and spectral bands
# that have been fitted using the k-distribution method

def h2o_gasbands():
    return [('h2o', '1'), 
            ('h2o', '2'),
            ('h2o', '3a'),
            ('h2o', '3b'),
            ('h2o', '3c'),
            ('h2o', '4'),
            ('h2o', '5'),
            ('h2o', '6'),
            ('h2o', '7'), 
            ('h2o', '8'),
            ('h2o', '9')]


def co2_gasbands():
    return [('co2', '3a'), ('co2', '3b'), ('co2', '3c'), ('co2', '9')]


def o3_gasbands():
    return [('o3', '5'), ('o3', '9')]


def n2o_gasbands():
    return [('n2o', '7')]


def ch4_gasbands():
    return [('ch4', '7')]

In [47]:
script()

h2o 1
h2o 2
h2o 3a
h2o 3b
h2o 3c
h2o 4
h2o 5
h2o 6
h2o 7
h2o 8
h2o 9
co2 3a
co2 3b
co2 3c
co2 9
o3 5
o3 9
n2o 7
ch4 7
